<h3>Tạo file train và test</h3>

In [90]:
import pandas as pd
import numpy as np

In [91]:
data = pd.read_csv('./input/Data.csv', encoding='utf-8')
data.head()

,comment,rate
0,Áo bao đẹp ạ!!,5
1,Tuyệt vời !,5
2,2day ao khong giong trong.,1
3,"Mùi thơm,bôi lên da mềm da.",5
4,"Vải đẹp, dày dặn.",5


In [92]:
# Gán nhãn dữ liệu #
label = []
for rate in data['rate']:
    if(rate == 5 or rate == 4): 
        a = "POS"
        label.append(a)
    elif (rate == 3):
        a = "NEU"
        label.append(a)
    else:
        a = "NEG"
        label.append(a)
data['label'] = label

data.drop(data.columns[1], axis = 1, inplace = True)

In [93]:
from vneseDataProcess import text_preprocess

<h5>Chuyển về dạng string</h5>

In [94]:
processed_comments = []
comment = data.comment
for item in comment:
    a = text_preprocess(item)
    processed_comments.append(a)

list_tuples2 = list(zip(processed_comments, data.label)) 
processed_data = pd.DataFrame(list_tuples2, columns = ['comment', 'label'])


<h5>Lọc bỏ empty cmt</h5>

In [95]:

nan_value = float("NaN")

processed_data.replace("", nan_value, inplace=True)

## dữ liệu dạng string
processed_data.dropna(subset= ['comment'], inplace=True)

<h5>Chuyển về dạng dic</h5>

In [96]:
from vneseDataProcess import list_to_dict_tokens

dic_comments = []
comment = processed_data.comment
for item in comment:
    item = item.split(' ')
    a = list_to_dict_tokens(item)
    dic_comments.append(a)

list_tuples = list(zip(dic_comments, processed_data.label)) 

# dữ liệu mà cmt là dạng từ điển
final_data = pd.DataFrame(list_tuples, columns = ['comment', 'label'])

In [97]:

data_label_count = final_data.groupby(['label']).count()['comment']
data_label_count

label
NEG     7523
NEU     5169
POS    22689
Name: comment, dtype: int64

<h5>Cắt dữ liệu</h5>

In [98]:
# data_neg = processed_data[processed_data['label'] == 'NEG' ]

# data_neu = processed_data[processed_data['label'] == 'NEU']

# data_pos = processed_data[processed_data['label'] == 'POS']

# data_neg = data_neg.iloc[:5170]
# data_pos = data_pos.iloc[:5170]

data_neg = final_data[final_data['label'] == 'NEG' ]

data_neu = final_data[final_data['label'] == 'NEU']

data_pos = final_data[final_data['label'] == 'POS']

data_neg = data_neg.iloc[:5170]
data_pos = data_pos.iloc[:5170]

final_data = pd.concat([data_pos, data_neu, data_neg])



In [99]:
final_data = final_data.sample(frac = 1)
final_data.groupby(['label']).count()['comment']


label
NEG    5170
NEU    5169
POS    5170
Name: comment, dtype: int64

In [103]:
final_data.head()

,comment,label
6243,"{'chỉ': 1, 'sợ': 1, 'không': 1, 'xinh': 1}",POS
21329,"{'đẹp': 1, 'bao': 1, 'bị': 1, 'rách': 1, 'tí': 1}",NEU
4225,"{'đẹp': 1, 'rất': 1, 'dễ_thương': 1}",POS
2977,{'ok': 1},POS
20151,"{'nam': 1, 'không': 2, 'hiệu_quả': 1}",NEU


<h5>Phân chia tập train và test (tỉ lệ 8-2)</h5>

In [101]:
from sklearn.model_selection import train_test_split
import pickle
comment = final_data['comment']
label = final_data['label']
x_train, x_test, y_train, y_test = train_test_split(comment, label, test_size=0.2)

# lưu file train
with open('x_train.pkl', 'wb') as f:
     pickle.dump(x_train, f)
with open('y_train.pkl', 'wb') as f:
     pickle.dump(y_train, f) 

#lưu file test
with open('x_test.pkl', 'wb') as f:
     pickle.dump(x_test, f)
with open('y_test.pkl', 'wb') as f:
     pickle.dump(y_test, f) 